In [ ]:
import pandas as pd
import os

class SingleAction():
    '''Создает класс действия и его параметрами.'''

    def __init__(self, new_action_name: str = None, exist_action_name: str = None) -> None:
        '''Создает действие с именем при инициализации класса.

        Функция создает эксземпляр класса SingleAction либо с новым именем и без параметров, либо с каким-то уже существующим именем и параметрами.
        Если задается переменная new_action_name, то реализуется первый случай. 
        Если задается переменная exist_action_name, то реализуется второй случай вне зависимости от значения первой переменной.
        Если новое имя совпадает с каким-то старым, то старое перезаписывается.

        Args:
            new_action_name (str): если нужно создать новое действие с новым именем (по умолчанию None)
            exist_action_name (str): если нужно загрузить уже существующее действие с известными параметрами (по умолчанию None)

        Returns:
            None
        '''
        if exist_action_name != None:
            self.read_from_csv(str(exist_action_name))
        elif new_action_name != None:
            self.action_name = str(new_action_name)
            with open('act_' + self.action_name, 'w') as f:
                pass 
        else:
            self.action_name = 'Default name'
            with open('act_' + self.action_name, 'w') as f:
                pass 
        

    def change_name(self, new_name: str) -> None:
        ''' Изменяет имя данного действия.'''
        self.action_name = str(new_name)
        self.save_to_csv()

    def add_param(self, name: str, value) -> None:
        '''Добавляет параметр для действия.

        Функция добавляет параметр и его значение в конец таблицы, если уже есть какие-то параметры. 
        Если таблица параметров пуста, то создается новая таблица.
        Формат таблицы - pd.DataFrame. 

        Args:
            name (str): название параметра
            value (any): значение параметра (любой тип данных)

        Returns:
            None
        '''
        new_param = pd.DataFrame({'Name': [str(name)], 'Value': [value]})
        if not hasattr(self, 'param_tabels'):
            self.param_tabels = new_param
        else:
            self.param_tabels = pd.concat([self.param_tabels, new_param], ignore_index=True)
        self.save_to_csv()

    def del_param(self, index: int) -> None:
        '''Удаляет параметр из таблицы.

        Функция удаляет определенный параметр из таблицы согласно его индексу по нумерации pd.DataFrame.
        Если индекс больше количества элементов или меньше или равен нулю, то возвращает ошибку.

        Args:
            index (int): индекс параметра, который будет удален

        Returns:
            None    
        '''
        if (index >= self.param_tabels.shape[0]) or (index <= 0):
            raise KeyError('Index out of range')
        self.param_tabels.drop(index, inplace = True)
        self.param_tabels.reset_index(drop = True, inplace = True)
        self.save_to_csv()

    # сохраняет действие в отдельный файл с приставкой "act"
    def save_to_csv(self) -> None:
        self.param_tabels.dropna(inplace=True)
        if not hasattr(self, 'param_tabels'):
            new_param = pd.DataFrame({'Name': [None], 'Value':[None]})
            self.param_tabels = new_param
        path_to_save = os.path.join(os.getcwd(), 'act_' + self.action_name)
        self.param_tabels.to_csv(path_to_save, index=False, sep=';')

    # считывает таблицу, проверка на считываемость, проверка на корректность внутреннего формата данных
    def read_from_csv(self, name: str) -> None:
        path_to_find = os.path.join(os.getcwd(), 'act_' + str(name))
        if os.path.exists(path_to_find):
            try:
                self.param_tabels = pd.read_csv(path_to_find, sep = ';')
                if self.param_tabels.shape[1] > 2:
                    raise ValueError(f'Action file contain more than 2 collums')
            except ValueError:
                print(f'Can not read \"{name}\" file')
            
            self.action_name = str(name)
        else:
            raise FileNotFoundError(f'Action \"{name}\" not found')

    # удаляет файл с сохраненным дейтсвием
    def del_csv(self, name: str) -> None:
        path_to_find = os.path.join(os.getcwd(), 'act_' + str(name))
        if os.path.exists(path_to_find):
            os.remove(path_to_find)
        else:
            raise FileNotFoundError(f'Action \"{name}\" not found')

In [67]:
'''a = SingleAction('deposition_copy')

a.add_param('temp in C', 200)
a.add_param('press in mBar', 1e-2)
a.add_param('pres', 1e2)
a.add_param('sdjkbf', 'lkasjvb')
a.param_tabels'''

"a = SingleAction('deposition_copy')\n\na.add_param('temp in C', 200)\na.add_param('press in mBar', 1e-2)\na.add_param('pres', 1e2)\na.add_param('sdjkbf', 'lkasjvb')\na.param_tabels"

In [68]:
a = SingleAction(exist_action_name='new')
#print(a.del_param.__doc__)


Can not read "new" file


In [69]:
pd.read_csv('act_new', sep = ';')

EmptyDataError: No columns to parse from file